In [96]:
import numpy as np
import pandas as pd
import jieba
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
import sklearn.naive_bayes as nb
import sklearn.metrics as sm
jieba.load_userdict('./data/htl_dict.txt')

In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
# 加载文件
data = pd.read_csv('./data/htl_all.csv')
data.head(3)

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。


In [99]:
data['label'].value_counts()

1    5322
0    2443
Name: label, dtype: int64

**样本均衡化**

In [100]:
# 下采样
# 由于data['label']是1到0的顺序
data = data.tail((data['label']==0).sum() * 2)

In [101]:
def func(item):
    # 分词，添加空格
    seg_list = jieba.cut(item)
    return ' '.join(seg_list)

data['new_review']=data['review'].apply(func)

In [102]:
data.head(3)

,label,review,new_review
2879,1,这个酒店住过了无数次了，客观的来说是个值得推荐的酒店，曾经的服务给我留下了无比深刻的印象，除...,这个 酒店 住 过 了 无数次 了 ， 客观 的 来说 是 个 值得 推荐 的 酒店 ， 曾...
2880,1,这次和同事一起订的房间，我在22楼，房间明显没有同事10楼的好，下次预定一定要指定楼层了。,这次 和 同事 一起 订 的 房间 ， 我 在 22 楼 ， 房间 明显 没有 同事 10 ...
2881,1,没有参加过评价，但感觉很多酒店的评价不客观。这次参与一下。此酒店我个人觉得在西安应属于性价比...,没有 参加 过 评价 ， 但 感觉 很多 酒店 的 评价 不 客观 。 这次 参与 一下 。...


In [103]:
import sklearn.feature_extraction.text as ft
# 整理输入集与输出集  TFIDF
cv = ft.CountVectorizer()
bow = cv.fit_transform(data.new_review)
tt = ft.TfidfTransformer()
tfidf = tt.fit_transform(bow)

# 拆分测试集与训练集
train_x, test_x, train_y, test_y = ms.train_test_split(
    tfidf, data.label, test_size=0.2, random_state=7
)

# 使用朴素贝叶斯
model = nb.MultinomialNB()

# 交叉验证
scores = ms.cross_val_score(
    model, test_x, test_y, cv=5, scoring='f1_weighted'
)
print(scores.mean())

# 训练模型
model.fit(train_x, train_y)

# 测试模型，评估模型
pred_test_y = model.predict(test_x)
print(sm.classification_report(test_y, pred_test_y))

0.8173838812692982
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       506
           1       0.86      0.86      0.86       472

    accuracy                           0.86       978
   macro avg       0.86      0.86      0.86       978
weighted avg       0.86      0.86      0.86       978



**查看置信概率**

In [104]:
# 新测试样本测试
new_test_data = ["这家酒店帮，环境棒，电视棒，空调不错，挺好的。",
                 "地板脏，空调漏水，再也不来了。",
                 "空调不太凉，早餐还行，但是服务差了点，总体一般。",
                 "我不喜欢，非常差！", "哪哪都好，房间大，视野开阔，环境非常好，下次还来！"]
new_test_data = pd.Series(new_test_data).apply(func)

# 将测试样本按照训练时的方式转换成TFIDF矩阵
bow = cv.transform(new_test_data)
new_test_data = tt.transform(bow)
pred_test_y = model.predict(new_test_data)

# 输出置信概率矩阵
pred_proba_y = model.predict_proba(new_test_data)
probas = pred_proba_y.max(axis=1)

for a, b, in zip(pred_test_y,probas):
    print(a,'->', b)


1 -> 0.5626072795519521
0 -> 0.8631187374183047
1 -> 0.6173843898266396
0 -> 0.8593407180490007
1 -> 0.9144705740529698
